In [ ]:
# Celda 1: Detectar y encolar sólo el nombre del .pbit
import time
import os
import config
import config_runtime
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from main import save_config_runtime, safe_print

class PbitHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.lower().endswith(".pbit"):
            return
        nombre = os.path.basename(event.src_path)
        if nombre in config_runtime.pbit_queue:
            return  # ya está en la cola
        config_runtime.pbit_queue.append(nombre)
        save_config_runtime()
        safe_print(f"📑 Cola actualizada: {config_runtime.pbit_queue}")

watch_folder = config.base_report_directory
observer = Observer()
observer.schedule(PbitHandler(), path=watch_folder, recursive=False)
observer.start()
safe_print(f"👀 Observando carpeta: {watch_folder} (Ctrl+C para parar)")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()
